In [ ]:
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as plt
import h5py
import cmath
import pandas as pd
from matplotlib.colors import ListedColormap, TwoSlopeNorm
from scipy.linalg import det, inv
from scipy.constants import h
from scipy.constants import k as k_B
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.colors import Normalize
from matplotlib.colors import LinearSegmentedColormap
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm
import squeezing_library as sqz
import json
from scipy.optimize import minimize
import csv

In [ ]:
def str_to_arr(filename, dataset):
    '''
    Converts the data strings in the csv file to numpy arrays
    '''
    csv.field_size_limit(10**7)
    modified_rows = []

    #Open the csv file
    with open(filename, newline='') as csvfile:
        reader = csv.DictReader(csvfile)

        #We need to remove the '[' charachter
        for row in reader:
            #We need to consider every column that is stored as string
            row['signal_pump_on_real'] = row['signal_pump_on_real'].replace(row['signal_pump_on_real'][0], "").replace(row['signal_pump_on_real'][-1], "")
            row["signal_pump_on_imag"] = row['signal_pump_on_imag'].replace(row['signal_pump_on_imag'][0], "").replace(row['signal_pump_on_imag'][-1], "")
            row['signal_pump_off_real'] = row['signal_pump_off_real'].replace(row['signal_pump_off_real'][0], "").replace(row['signal_pump_off_real'][-1], "")
            row['signal_pump_off_imag'] = row['signal_pump_off_imag'].replace(row['signal_pump_off_imag'][0], "").replace(row['signal_pump_off_imag'][-1], "")
            row['idler_pump_on_real'] = row['idler_pump_on_real'].replace(row['idler_pump_on_real'][0], "").replace(row['idler_pump_on_real'][-1], "")
            row["idler_pump_on_imag"] = row['idler_pump_on_imag'].replace(row['idler_pump_on_imag'][0], "").replace(row['idler_pump_on_imag'][-1], "")
            row['idler_pump_off_real'] = row['idler_pump_off_real'].replace(row['idler_pump_off_real'][0], "").replace(row['idler_pump_off_real'][-1], "")
            row['idler_pump_off_imag'] = row['idler_pump_off_imag'].replace(row['idler_pump_off_imag'][0], "").replace(row['idler_pump_off_imag'][-1], "")
            modified_rows.append(row)

    #New rows without the brackets
    dataset["signal_pump_on_real"] = [row["signal_pump_on_real"] for row in modified_rows]
    dataset["signal_pump_on_imag"] = [row["signal_pump_on_imag"] for row in modified_rows]
    dataset["signal_pump_off_real"] = [row["signal_pump_off_real"] for row in modified_rows]
    dataset["signal_pump_off_imag"] = [row["signal_pump_off_imag"] for row in modified_rows]
    dataset["idler_pump_on_real"] = [row["idler_pump_on_real"] for row in modified_rows]
    dataset["idler_pump_off_real"] = [row["idler_pump_off_real"] for row in modified_rows]
    dataset["idler_pump_on_imag"] = [row["idler_pump_on_imag"] for row in modified_rows]
    dataset["idler_pump_off_imag"] = [row["idler_pump_off_imag"] for row in modified_rows]

    #Convert the strings into float
    dataset['signal_pump_on_real'] = dataset['signal_pump_on_real'].apply(lambda x: np.array([float(num) for num in x.split(',')]))
    dataset['signal_pump_on_imag'] = dataset['signal_pump_on_imag'].apply(lambda x: np.array([float(num) for num in x.split(',')]))
    dataset['signal_pump_off_real'] = dataset['signal_pump_off_real'].apply(lambda x: np.array([float(num) for num in x.split(',')]))
    dataset['signal_pump_off_imag'] = dataset['signal_pump_off_imag'].apply(lambda x: np.array([float(num) for num in x.split(',')]))
    dataset['idler_pump_on_real'] = dataset['idler_pump_on_real'].apply(lambda x: np.array([float(num) for num in x.split(',')]))
    dataset['idler_pump_on_imag'] = dataset['idler_pump_on_imag'].apply(lambda x: np.array([float(num) for num in x.split(',')]))
    dataset['idler_pump_off_real'] = dataset['idler_pump_off_real'].apply(lambda x: np.array([float(num) for num in x.split(',')]))
    dataset['idler_pump_off_imag'] = dataset['idler_pump_off_imag'].apply(lambda x: np.array([float(num) for num in x.split(',')]))

    return dataset

In [ ]:
# Funzione per applicare il modulo 2pi
def modulo_2pi(x):
    return (x + np.pi) % (2 * np.pi) - np.pi  # Modulo 2pi, risultato nell'intervallo [-pi, pi]

In [ ]:
# Funzione per applicare il modulo 2pi
def modulo_pi(x):
    return (x + np.pi/2) % (np.pi) - np.pi/2  # Modulo 2pi, risultato nell'intervallo [-pi, pi]

In [ ]:
# Funzione per applicare il modulo pi/4
def modulo_pi_4(x):
    return (x + np.pi / 4) % (np.pi / 2) - np.pi / 4  # Modulo pi/4, risultato nell'intervallo [-pi/4, pi/4]


In [ ]:
# Funzione per applicare il modulo pi/4
def modulo_pi_2(x):
    return (x + np.pi / 2) % (np.pi ) - np.pi / 2  # Modulo pi/4, risultato nell'intervallo [-pi/4, pi/4]

In [ ]:
 # for each quadrature, using iq function, we reconstruct sqz.iq quadrature at dut output level
def iq_at_dut(R_on: np.ndarray,
              R_off: np.ndarray,
              volt: bool, #true if the output needs to be in volts
              f,
              G,
              tau,
              Z): #parameters for the sqz.iq function, list (or arrays) of 2 elements
    R_on_dut = []
    R_off_dut = []
    if volt:
        for p in range(4):
            if p < 2:
                R_on_dut.append(sqz.iq(R_on[p]*1.6, f[0], G[0], tau, Z))
                R_off_dut.append(sqz.iq(R_off[p]*1.6, f[0], G[0], tau, Z))
            else:
                R_on_dut.append(sqz.iq(R_on[p]*1.6, f[1], G[1], tau, Z))
                R_off_dut.append(sqz.iq(R_off[p]*1.6, f[1], G[1], tau, Z))
    else:
        for p in range(4):
            if p < 2:
                R_on_dut.append(sqz.iq(R_on[p], f[0], G[0], tau, Z))
                R_off_dut.append(sqz.iq(R_off[p], f[0], G[0], tau, Z))
            else:
                R_on_dut.append(sqz.iq(R_on[p], f[1], G[1], tau, Z))
                R_off_dut.append(sqz.iq(R_off[p], f[1], G[1], tau, Z))

    return R_on_dut, R_off_dut

In [ ]:
#2d histograms
def new_hist(R_on, R_off, nbins: int, bin_edges: np.ndarray, norm, x_mode1: bool, p_mode1: bool, x_mode2: bool, p_mode2: bool, tick):
    # Map of modes (x and p) for each case
    modes = [(x_mode1, '$x_{s}$', '$\\bar{x}_{s}$', R_on[0], R_off[0]), 
             (p_mode1, '$p_{s}$', '$\\bar{p}_{s}$', R_on[1], R_off[1]),
             (x_mode2, '$x_{i}$', '$\\bar{x}_{i}$', R_on[2], R_off[2]),
             (p_mode2, '$p_{i}$', '$\\bar{p}_{i}$', R_on[3], R_off[3])]
    
    # Initialize the variables
    #alpha = [x_mode1, p_mode1, x_mode2, p_mode2]
    c = 0
    x, y = [], []
    #x_mean_on, x_mean_off, y_mean_on, y_mean_off = 0, 0, 0, 0

    # For loop to check which modes to use for 2d histogram
    for mode, name, mean_name, R_on_mode, R_off_mode in modes:
        if mode:
            if c == 0:
                x_name = name
                #x_mean_name = mean_name
                #x_mean_on, x_mean_off = np.mean(R_on_mode), np.mean(R_off_mode)
                x = np.concatenate([R_on_mode, R_off_mode])
                c = 1
            else:
                y_name = name
                #y_mean_name = mean_name
                #y_mean_on, y_mean_off = np.mean(R_on_mode), np.mean(R_off_mode)
                y = np.concatenate([R_on_mode, R_off_mode])
    
    # Create the histogram
    g = plt.hist2d(
        x, 
        y, 
        weights=np.concatenate([np.ones(len(R_on[0]))/(len(R_on[0]))*100, -np.ones(len(R_on[0]))/(len(R_on[0]))*100]),
        bins=(nbins, nbins), 
        range=(bin_edges, bin_edges), 
        cmap='bwr', 
        norm=norm
    )

    # Axes settings
    plt.xlabel(x_name, fontsize=15)
    plt.ylabel(y_name, fontsize=15)
    plt.tick_params(axis='both', labelsize=12)
    plt.xticks(np.arange(bin_edges[0], bin_edges[1] + 1, tick))
    plt.yticks(np.arange(bin_edges[0], bin_edges[1] + 1, tick))

    return g

#cosa fa di diverso da prima?
#Abbiamo mappato le variabili (x_mode1, p_mode1, etc.) in una lista chiamata modes, 
#che contiene tuple con i dati e le etichette associate. Questo consente di evitare la duplicazione di codice 
#e facilita la gestione delle modalità.
#Utilizziamo np.concatenate per unire gli array R_on_mode e R_off_mode invece di usare 
#[*R_on[i], *R_off[i]], che è più efficiente e più chiaro.

This is the data analysis of a two mode squeezing experiment in a Traveling Wave Parametric Amplifier.

Import data
-----------------

In [ ]:
import pandas as pd
import xarray as xr

# Carica il file CSV in un DataFrame di pandas
#df = pd.read_csv(r"I:\Drive condivisi\SuperQuElectronics\Simulations, Code and Data Analysis - MADE by SQE\Python\QCS\Data Analysis\CARTHAGO SW2401010B - Squeezing analysis3.csv")
df = pd.read_csv(r"Data/CARTHAGO SW2401010B - Squeezing analysis3.csv")
# Converti il DataFrame in un xarray DataArray o Dataset
# Supponiamo che tu voglia creare un Dataset con le colonne del DataFrame
# Le righe del CSV diventeranno le dimensioni
#data_array = xr.DataArray(df.values, coords=[df.index, df.columns], dims=["row", "col"])

# Oppure, se vuoi creare un Dataset con più variabili (colonne)
#dataset = xr.Dataset.from_dataframe(df)

In [ ]:
df

In [ ]:
frequencies = np.array((df["Pump frequency"]))
pump_dig_phase = np.array(np.radians(df["pump_dig_phase"]))

In [ ]:
print(pump_dig_phase)

In [ ]:
print(frequencies)

In [ ]:
df = str_to_arr(r"Data/CARTHAGO SW2401010B - Squeezing analysis3.csv", df)

In [ ]:
print(type(df["signal_pump_on_real"].iloc[0]))
print(len(df["signal_pump_on_real"].iloc[0]))
df

Searching angle $\theta$ which maximizes squeezing

In [ ]:
theta_array = np.zeros(len(frequencies))
file_csv = 'gains.csv'
tau = 6e-6
Z = 50

for i in range(len(frequencies)):
    I_s_off_or = df["signal_pump_off_real"].iloc[i]
    Q_s_off_or = df["signal_pump_off_imag"].iloc[i]
    I_s_on_or = df["signal_pump_on_real"].iloc[i]
    Q_s_on_or = df["signal_pump_on_imag"].iloc[i]
    I_i_on_or = df["idler_pump_on_real"].iloc[i]
    Q_i_on_or = df["idler_pump_on_imag"].iloc[i]
    I_i_off_or = df["idler_pump_off_real"].iloc[i]
    Q_i_off_or = df["idler_pump_off_imag"].iloc[i]

 

    R_on = np.stack([I_s_on_or, Q_s_on_or, I_i_on_or, Q_i_on_or])
    R_off = np.stack([I_s_off_or, Q_s_off_or, I_i_off_or, Q_i_off_or])

    fs = frequencies[i] - 275e6
    fi = frequencies[i] + 275e6
    G_s = sqz.db_to_linear(sqz.gain_finder(file_csv, fs)[1]+10)  #74 dB 75
    G_i = sqz.db_to_linear(sqz.gain_finder(file_csv, fi)[1]+10) #74 dB 75
    
    #f = [fs, fi]
    #G = [G_s, G_i]

    #change with iq_at_dut function
    #perchè qui moltiplico subito per 1.6 e poi trovo il minimo della varianza?
    #R_on, R_off = iq_at_dut(R_on, R_off, True, f, G, tau, Z)
    for p in range(4):       # for each quadrature, using iq function, we reconstruct sqz.iq quadrature at dut output level
        if p < 2:
            R_on[p]= sqz.iq(R_on[p]*1.6, fs, G_s, tau, Z)
            R_off[p]= sqz.iq(R_off[p]*1.6, fs, G_s, tau, Z)
        else:
            R_on[p]= sqz.iq(R_on[p]*1.6, fi, G_i, tau, Z)
            R_off[p]= sqz.iq(R_off[p]*1.6, fi, G_i, tau, Z)

    result = minimize(lambda theta: sqz.calculate_var1(theta,R_on,R_off), x0=[1], bounds=[(-np.pi, np.pi)])
    #print(result.x[0])
    theta_array[i] = result.x[0]

In [ ]:
modulo_pi(theta_array)

In [ ]:
modulo_pi(pump_dig_phase)

In [ ]:
modulo_pi(2*theta_array-2*pump_dig_phase)

Angles of delay of counterpump

In [ ]:
str = ["exp_1DNC_1DIG_2024-11-27_12-41", "exp_1DNC_1DIG_2024-11-27_12-43", "exp_1DNC_1DIG_2024-11-27_12-46", "exp_1DNC_1DIG_2024-11-27_12-48", "exp_1DNC_1DIG_2024-11-27_12-51", "exp_1DNC_1DIG_2024-11-27_12-53", "exp_1DNC_1DIG_2024-11-27_12-56", "exp_1DNC_1DIG_2024-11-27_12-58", "exp_1DNC_1DIG_2024-11-27_13-01", "exp_1DNC_1DIG_2024-11-27_13-03", "exp_1DNC_1DIG_2024-11-27_13-06", "exp_1DNC_1DIG_2024-11-27_13-08", "exp_1DNC_1DIG_2024-11-27_13-11", "exp_1DNC_1DIG_2024-11-27_13-13", "exp_1DNC_1DIG_2024-11-27_13-15"]
phases = np.zeros(15)

for i in range(15):
    file_demod = h5py.File(f"F:\Shared drives\SuperQuElectronics\Measurements\\2024\QCS_API\Squeezing_202411\\{str[i]}.hdf5")
    list(file_demod.attrs.keys())
    file_demod.attrs["FPGAPostprocessing"]
    chan = file_demod.attrs["Program"]
    phases[i] = sqz.extract_values_from_list(sqz.extract_info(chan, 'Phase'))[1]


phases

In [ ]:
A = modulo_pi(((2*pump_dig_phase) - ((2*(theta_array)))))
A

In [ ]:
A%np.pi

In [ ]:
plt.plot(frequencies,A)
plt.show()

In [ ]:
print(A)

In [ ]:
import numpy as np
from scipy.linalg import solve

a = np.array([60,-2,-3,+1,+2,+3])
phi = np.zeros((len(a),16))

for k in range(len(a)):
    # Inizializzazione
    noto = np.zeros(16)  # Vettore dei termini noti
    rand = np.random.uniform(low=-0.1, high=0.1, size=16)
    vettore = np.full(16, 1)
    pi = np.pi  # Pi greco per il modulo 2pi
    delta_phi = a[k]

    # Inizializza la matrice A (16x16) e il vettore b
    matrix = np.zeros((16, 16))

    # Popolare la matrice seguendo le regole
    for i in range(15):  # Le prime 14 righe
        matrix[i, i] = -1    # Elemento in posizione i
        matrix[i, i+1] = 1   # Elemento in posizione i+1

        noto[i] = rand[i]
        #noto[i] = -A[i]  # Imposta i termini noti come -A[i]  #rand[i]#
        #noto[i] = vettore[i]  # Imposta i termini noti come -A[i]  #rand[i]#

    # L'ultima riga deve essere tutta uguale a 1
    matrix[15, :] = 1
    noto[15] = delta_phi#+2*np.pi*80  # Imposta l'ultimo termine noto

    soluzioni = solve(matrix, noto)

    # Estrai i valori delle variabili
    phi[k] = soluzioni




# Stampa la matrice e il vettore dei termini noti
print("Matrice A:")
print(matrix)
print("\nVettore dei termini noti:")
print(noto)



# Stampa le soluzioni
print("\nSoluzioni:")
print(phi)


In [ ]:
import numpy as np
from scipy.linalg import solve

a = np.array([-3,-2,-1,0,+1,+2,+3])
phi = np.zeros((len(a),16))

for k in range(len(a)):
    # Inizializzazione
    noto = np.zeros(16)  # Vettore dei termini noti
    rand = np.random.uniform(low=-np.pi/4, high=np.pi/4, size=16)
    vettore = np.full(16, 0)
    pi = np.pi  # Pi greco per il modulo 2pi
    delta_phi = a[k]

    # Inizializza la matrice A (16x16) e il vettore b
    matrix = np.zeros((16, 16))

    # Popolare la matrice seguendo le regole
    for i in range(15):  # Le prime 14 righe
        matrix[i, i] = -1    # Elemento in posizione i
        matrix[i, i+1] = 1   # Elemento in posizione i+1

        #noto[i] = rand[i]
        #noto[i] = -A[i]  # Imposta i termini noti come -A[i]  #rand[i]#
        noto[i] = vettore[i]  # Imposta i termini noti come -A[i]  #rand[i]#

    # L'ultima riga deve essere tutta uguale a 1
    matrix[15, 0] = 1
    matrix[15, 15] = -1
    matrix[15, 14] = -1
    #matrix[15, 13] = 1
    noto[15] = delta_phi#+2*np.pi*80  # Imposta l'ultimo termine noto

    soluzioni = solve(matrix, noto)

    # Estrai i valori delle variabili
    phi[k] = soluzioni




# Stampa la matrice e il vettore dei termini noti
print("Matrice A:")
print(matrix)
print("\nVettore dei termini noti:")
print(noto)



# Stampa le soluzioni
print("\nSoluzioni:")
print(phi)


In [ ]:
plt.plot(np.linspace(4.4, 8.8, 16), (phi[0]), linestyle='', marker='o')
plt.show()

In [ ]:
plt.plot(np.linspace(4.4, 8.8, 16), (phi[0]%(2*np.pi)), linestyle='', marker='o')
plt.show()

In [ ]:
new_array = np.zeros((len(a), 16))  # Array di zeri (dimensione len(a) x 16)
for i in range(len(a)):
    new_array[i] = np.cumsum(phi[i])

# Aggiungere un elemento nullo all'inizio di ogni riga
new_array = np.hstack([np.zeros((len(a), 1)), new_array])

In [ ]:
np.radians(4000)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

freq = []
for i in range(0, 17):
    freq.append(f"$\mathbf{{f_{{{i}}}}}$")

mpl.rcParams['font.family'] = 'Times New Roman'
mpl.rcParams['font.serif'] = ['Times New Roman']
mpl.rcParams['mathtext.fontset'] = 'stix'  # per il supporto dei caratteri matematici
#color = ["red","blue","grey"]
for i in range(len(a)):
    plt.plot(freq, new_array[i], label=f"a/rad = {a[i]}", marker = ".")
# plt.plot(np.linspace(4.4, 8.8, 16), new_array[1], label=f"a/rad = {a[1]}", marker = ".")
# plt.plot(np.linspace(4.4, 8.8, 16), new_array[2], label=f"a/rad = {a[2]}", marker = ".")
plt.xlabel("Frequency", fontsize=20, fontweight='bold', color='black')
plt.ylabel("$\mathbf{(\phi_{f}-\phi_{0})}$/rad", fontsize=20, fontweight='bold', color='black')
ticks_pari = np.arange(0, 17, 2)  # Ticks solo per gli indici pari
plt.xticks(ticks_pari,fontsize=18, fontweight='bold', color='black') 
plt.yticks(fontsize=15, fontweight='bold', color='black') 
plt.legend()
plt.show()

In [ ]:
i = 0

In [ ]:
theta_array[i]

In [ ]:
A[i]

In [ ]:
theta_array

In [ ]:
I_s_off_or = df["signal_pump_off_real"].iloc[i]
Q_s_off_or = df["signal_pump_off_imag"].iloc[i]
I_s_on_or = df["signal_pump_on_real"].iloc[i]
Q_s_on_or = df["signal_pump_on_imag"].iloc[i]
I_i_on_or = df["idler_pump_on_real"].iloc[i]
Q_i_on_or = df["idler_pump_on_imag"].iloc[i]
I_i_off_or = df["idler_pump_off_real"].iloc[i]
Q_i_off_or = df["idler_pump_off_imag"].iloc[i]

In [ ]:
# manipolo dati ruotandoli nle piano iq
comp_s_off = I_s_off_or + 1j*Q_s_off_or
comp_s_on = I_s_on_or + 1j*Q_s_on_or
comp_i_on = I_i_on_or + 1j*Q_i_on_or
comp_i_off = I_i_off_or + 1j*Q_i_off_or

#theta = 0
theta = theta_array[i]+np.pi
#theta = -pump_dig_phase[i]
delta_theta = np.radians(0)
rot = np.exp(1j * theta)  # e^(i*theta)
delta_rot = np.exp(1j * delta_theta)
comp_s_off_rot = comp_s_off*rot
comp_s_on_rot = comp_s_on*rot
comp_i_off_rot = comp_i_off*rot*delta_rot
comp_i_on_rot = comp_i_on*rot*delta_rot

I_s_off = np.array(comp_s_off_rot.real)
Q_s_off = np.array(comp_s_off_rot.imag)

I_s_on = np.array(comp_s_on_rot.real)
Q_s_on = np.array(comp_s_on_rot.imag)

I_i_on = np.array(comp_i_on_rot.real)
Q_i_on = np.array(comp_i_on_rot.imag)

I_i_off = np.array(comp_i_off_rot.real)
Q_i_off = np.array(comp_i_off_rot.imag)

R_on = np.stack([I_s_on, Q_s_on, I_i_on, Q_i_on])
R_off = np.stack([I_s_off, Q_s_off, I_i_off, Q_i_off])

In [ ]:
#sostituire con
#R_on, R_off = iq_at_dut(R_on, R_off, True, f, G, tau, Z)
#per un codice più compatto
for p in range(4):       # for each quadrature, using iq function, we reconstruct sqz.iq quadrature at dut output level
    if p < 2:
        R_on[p]= sqz.iq(R_on[p]*1.6, fs, G_s, tau, Z)
        R_off[p]= sqz.iq(R_off[p]*1.6, fs, G_s, tau, Z)
    else:
        R_on[p]= sqz.iq(R_on[p]*1.6, fi, G_i, tau, Z)
        R_off[p]= sqz.iq(R_off[p]*1.6, fi, G_i, tau, Z)

In [ ]:
bin_edges = (-5, 5)  
nbins=500
vmin = -50  # Limite inferiore
vmax = 50  # Limite superiore
norm = TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)  # Crea l'istanza di normalizzazione con vmin e vmax
tick = 5

fig = plt.figure(figsize=(8, 7.5)) # Crea il colorplot dell'istogramma 2D

# x_mode1: bool, p_mode1: bool, x_mode2: bool, p_mode2: bool
plt.subplot(2, 2, 1)  # 1 row, 2 columns, 2nd subplot
plt.hist2d(R_on[0], R_on[1], bins=(nbins, nbins), range=(bin_edges,bin_edges), cmap='bwr', norm=norm,)
plt.xlabel("$x_{son}$", fontsize=15)
plt.ylabel("$p_{son}$", fontsize=15)
plt.subplot(2, 2, 2)  # 1 row, 2 columns, 2nd subplot
counts, xedges, yedges, im = plt.hist2d(R_on[2], R_on[3], bins=(nbins, nbins), range=(bin_edges,bin_edges), cmap='bwr', norm=norm,)
plt.xlabel("$x_{ion}$", fontsize=15)
plt.ylabel("$p_{ion}$", fontsize=15)
plt.subplot(2, 2, 3)  # 1 row, 2 columns, 2nd subplot
counts, xedges, yedges, im = plt.hist2d(R_off[0], R_off[1], bins=(nbins, nbins), range=(bin_edges,bin_edges), cmap='bwr', norm=norm,)
plt.xlabel("$x_{soff}$", fontsize=15)
plt.ylabel("$p_{soff}$", fontsize=15)
plt.subplot(2, 2, 4)  # 1 row, 2 columns, 1st subplot
counts, xedges, yedges, im = plt.hist2d(R_off[2], R_off[3], bins=(nbins, nbins), range=(bin_edges,bin_edges), cmap='bwr', norm=norm,)
plt.xlabel("$x_{ioff}$", fontsize=15)
plt.ylabel("$p_{ioff}$", fontsize=15)
# Aggiunta di una colorbar personalizzata
cbar_ax = fig.add_axes([1.05, 0.1, 0.02, 0.8])  # [x0, y0, width, height]
cbar = fig.colorbar(im, cax=cbar_ax)
cbar.set_label('Counts', fontsize=15)
cbar.ax.tick_params(labelsize=12)
plt.tight_layout()
plt.show()

Pump on - pump off istograms
-----------------

In [ ]:
bin_edges = (-5, 5)  
nbins=200
vmin = -0.01  # Limite inferiore
vmax = 0.01  # Limite superiore
norm = TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)  # Crea l'istanza di normalizzazione con vmin e vmax
tick = 5

fig = plt.figure(figsize=(12, 7.5)) # Crea il colorplot dell'istogramma 2D

# x_mode1: bool, p_mode1: bool, x_mode2: bool, p_mode2: bool
plt.subplot(2, 3, 1)  # 1 row, 2 columns, 2nd subplot
new_hist(R_on, R_off, nbins, bin_edges, norm, True, True, False, False,tick)
plt.subplot(2, 3, 2)  # 1 row, 2 columns, 2nd subplot
new_hist(R_on, R_off, nbins, bin_edges, norm, False, False, True, True,tick)
plt.subplot(2, 3, 3)  # 1 row, 2 columns, 2nd subplot
new_hist(R_on, R_off, nbins, bin_edges, norm, True, False, True, False,tick)
plt.subplot(2, 3, 4)  # 1 row, 2 columns, 1st subplot
new_hist(R_on, R_off, nbins, bin_edges, norm, False, True, True, False,tick)
plt.subplot(2, 3, 5)  # 1 row, 2 columns, 2nd subplot
new_hist(R_on, R_off, nbins, bin_edges, norm, True, False, False, True,tick)
plt.subplot(2, 3, 6)  # 1 row, 2 columns, 2nd subplot
g = new_hist(R_on, R_off, nbins, bin_edges, norm, False, True, False, True,tick)
# Aggiunta di una colorbar personalizzata
cbar_ax = fig.add_axes([1.05, 0.1, 0.02, 0.8])  # [x0, y0, width, height]
cbar = fig.colorbar(g[3], cax=cbar_ax)
cbar.set_label('Counts on - counts off [%]', fontsize=15)
cbar.ax.tick_params(labelsize=12)
plt.tight_layout()
plt.show()

In [ ]:
on = np.sqrt(R_on[0]**2 + R_on[1]**2)
# Calcola la deviazione standard delle distanze
std_deviation_on = np.std(on)

off = np.sqrt(R_off[0]**2 + R_off[1]**2)
# Calcola la deviazione standard delle distanze
std_deviation_off = np.std(off)

gain = std_deviation_on/std_deviation_off
gain

In [ ]:
on = np.sqrt(R_on[2]**2 + R_on[3]**2)
# Calcola la deviazione standard delle distanze
std_deviation_on = np.std(on)

off = np.sqrt(R_off[2]**2 + R_off[3]**2)
# Calcola la deviazione standard delle distanze
std_deviation_off = np.std(off)

gain = std_deviation_on/std_deviation_off
gain

Thermal States Gaussianity
---------------------

In [ ]:
# x_mode1: bool, p_mode1: bool, x_mode2: bool, p_mode2: bool
par = sqz.gaussianity_proj(R_off, True, False, True, False)
sigma = par[2]
print(sigma)

Covariance matrix
-----------------

$\sigma^{meas,on} = \sigma _{|TMS>} + \sigma _{thermal}$


$\sigma^{meas,off} = \sigma _{|0>} + \sigma _{thermal}$


$\sigma _{|TMS>} = \sigma^{meas,on} - \sigma^{meas,off} + \sigma _{|0>}$

Reference: https://arxiv.org/pdf/2111.03696

In [ ]:
sigma_meas_on = sqz.cov_matrix(R_on)
sigma_meas_off = sqz.cov_matrix(R_off)
identita = 0.25*np.eye(4)
sigma = sigma_meas_on-sigma_meas_off+identita
sigma

In [ ]:
labels = ["$x_{s}$", "$p_{s}$", "$x_{i}$", "$p_{i}$"]  # Etichette degli assi
x, y = np.meshgrid(np.arange(sigma.shape[0]), np.arange(sigma.shape[1]))  # Creazione della griglia per l'istogramma 2D
x = x.ravel()
y = y.ravel()
z = np.zeros_like(x)
dz = sigma.ravel()   # Altezze delle barre (elementi della matrice di covarianza)
dx = dy = 0.8  # Dimensioni delle barre
colors = np.where(dz >= 0, 'red', 'blue')  # Creazione di una mappa di colori personalizzata per positivo/negativo
alpha_values = np.where(dz >= 0, 0.8, 0.8)
fig_cov_matrix = plt.figure(figsize=(8, 8))  # Creazione del plot 3D
fig_cov_matrix.patch.set_alpha(0)  # Sfondo trasparente della figura
ax = fig_cov_matrix.add_subplot(111, projection='3d')

#bar_plot = ax.bar3d(x, y, z, dx, dy, dz, shade=True, color=colors, alpha = 1 , zsort='average')  # Grafico a barre 3D

for xi, yi, zi, dzi, color, alpha in zip(x, y, z, dz, colors, alpha_values):
    ax.bar3d(xi, yi, zi, dx, dy, dzi, shade=True, color=color, alpha=alpha, edgecolor='k',linewidth=1 ,zsort='average')



ax.view_init(elev=30, azim=-140)  # Impostazione dell'angolo di vista
#ax.set_zlabel('Covariance')  # Etichette degli assi
ax.set_xticks(np.arange(len(labels)))
ax.set_yticks(np.arange(len(labels)))
ax.set_xticklabels(labels)
ax.set_yticklabels(labels)
cmap = LinearSegmentedColormap.from_list('red_blue', ['blue', 'red'], N=2)  # Creazione di una mappa di colori discreta (rosso/blu)
norm = Normalize(vmin=-1, vmax=1)
sm = cm.ScalarMappable(cmap=cmap, norm=norm)   # Aggiunta della barra colori
sm.set_array([])  # Necessario per evitare avvisi
#plt.title("Covariance Matrix")  # Mostra il grafico
plt.show()

Squeezing
----------

Paragraph 4.5.2 Eichler PhD thesis

In [ ]:
# X- = xs - xi
# Var(X−Y)=Var(X)+Var(Y)−2⋅Cov(X,Y)
var1 = sigma[0,0] + sigma[2,2] + (2*sigma[2,0])
var1

# P+ = ps + pi
# Var(X+Y)=Var(X)+Var(Y)+2⋅Cov(X,Y)
var2 = sigma[1,1] + sigma[3,3] + (2*sigma[1,3])
var2

sqz_x = 10*np.log((var1)/0.5)
sqz_p = 10*np.log((var2)/0.5)

print(sqz_x)
print(sqz_p)

Logarithmic negativity
---------------

In [ ]:
print(sqz.log_neg(sigma))

Wigner function reconstruction
----------------------

For Gaussian states,

$W(\alpha) = \frac{1}{4\pi^2 \sqrt{\text{det}(V)}} \exp \left( -\frac{1}{2} \alpha V^{-1} \alpha^T \right)$

Reference: https://journals.aps.org/prl/pdf/10.1103/PhysRevLett.107.113601


In [ ]:
X_grid = Y_grid = np.linspace(-3, 3, 100)  # (X,Y) grid on which we compute Wigner function
colors = [(0., 'darkblue'), (0.2, 'blue'), (0.4, 'skyblue'), (0.6, 'yellow'), (0.8, 'red'), (1., 'brown')] 
cmap = LinearSegmentedColormap.from_list('custom_cmap', colors)
#cmap = 'viridis'
vmin = 0.0    # colormap limits
vmax= 0.23
v = np.linspace(vmin,vmax,6)
vcenter = vmax/2
norm = TwoSlopeNorm(vmin=vmin,vcenter = vcenter, vmax=vmax)  # Crea l'istanza di normalizzazione con vmin e vmax
up_ins = 2   # iset limits
low_ins = -2

fig = plt.figure(figsize=(12, 8))

plt.subplot(2, 3, 1)    # ideal vacuum state
xs,ps,xi,pi = True, True, False, False   # whether to project onto this dimension or not
W_values_vac = sqz.wigner_proj(identita, X_grid, Y_grid, xs, ps, xi, pi)   # wigner function projection on the desired dimesnions on the grid
sqz.wigner_plot('Ideal vacuum state', W_values_vac, cmap, X_grid, Y_grid, vmin, vmax, xs,ps,xi,pi)   # wigner function colorplot
ax_inset = sqz.inset_constructor_2(low_ins, up_ins)
sqz.level_curve(ax_inset,identita, xs,ps,xi,pi,"blue",3)

plt.subplot(2, 3, 4)   # ideal thermal state
T = 0.15  # temperature of thermal state
V_the = (2*sqz.N(fs,T)+1)*identita
xs,ps,xi,pi = True, True, False, False
W_values_the = sqz.wigner_proj(V_the ,X_grid, Y_grid, xs,ps,xi,pi)
g = sqz.wigner_plot('Ideal thermal state', W_values_the, cmap, X_grid, Y_grid, vmin, vmax,xs,ps,xi,pi)
#sqz.inset_constructor_2(X_grid, Y_grid,V_the,xs,ps,xi,pi, low_ins, up_ins, 'green',3)
ax_inset = sqz.inset_constructor_2(low_ins, up_ins)
sqz.level_curve(ax_inset,V_the, xs,ps,xi,pi,"green",3)

plt.subplot(2, 3, 2)  # Reconstructed state xs ps
xs,ps,xi,pi = True, True, False, False
V_the = (2*sqz.N(fs,T)+1)*identita
W_values = sqz.wigner_proj(sigma,X_grid, Y_grid, xs,ps,xi,pi)
sqz.wigner_plot('Reconstructed state', W_values, cmap, X_grid, Y_grid, vmin, vmax,xs,ps,xi,pi)
ax_inset = sqz.inset_constructor_2(low_ins, up_ins)
sqz.level_curve(ax_inset,identita, xs,ps,xi,pi,"blue",3)
sqz.level_curve(ax_inset,V_the, xs,ps,xi,pi,"green",3)
sqz.level_curve(ax_inset,sigma, xs,ps,xi,pi,"red",3)

plt.subplot(2, 3, 5)  # Reconstructed state xi pi
xs,ps,xi,pi = False, False, True, True
V_the = (2*sqz.N(fi,T)+1)*identita
W_values = sqz.wigner_proj(sigma, X_grid, Y_grid,xs,ps,xi,pi)
sqz.wigner_plot('Reconstructed state', W_values, cmap, X_grid, Y_grid, vmin, vmax,xs,ps,xi,pi)
ax_inset = sqz.inset_constructor_2(low_ins, up_ins)
sqz.level_curve(ax_inset,identita, xs,ps,xi,pi,"blue",3)
sqz.level_curve(ax_inset,V_the, xs,ps,xi,pi,"green",3)
sqz.level_curve(ax_inset,sigma, xs,ps,xi,pi,"red",3)

plt.subplot(2, 3, 3)  # Reconstructed state xs xi
xs,ps,xi,pi = True, False, True, False
V_the = (2*sqz.N(fs,T)+1)*identita
W_values = sqz.wigner_proj(sigma, X_grid, Y_grid,xs,ps,xi,pi)
sqz.wigner_plot('Reconstructed state', W_values, cmap,X_grid, Y_grid, vmin, vmax,xs,ps,xi,pi)
ax_inset = sqz.inset_constructor_2(low_ins, up_ins)
sqz.level_curve(ax_inset,identita, xs,ps,xi,pi,"blue",3)
sqz.level_curve(ax_inset,V_the, xs,ps,xi,pi,"green",3)
sqz.level_curve(ax_inset,sigma, xs,ps,xi,pi,"red",3)

plt.subplot(2, 3, 6)  # Reconstructed state ps pi
xs,ps,xi,pi = False, True, False, True
W_values = sqz.wigner_proj(sigma, X_grid, Y_grid,xs,ps,xi,pi)
sqz.wigner_plot('Reconstructed state', W_values, cmap, X_grid, Y_grid, vmin, vmax,xs,ps,xi,pi)
ax_inset = sqz.inset_constructor_2(low_ins, up_ins)
sqz.level_curve(ax_inset,identita, xs,ps,xi,pi,"blue",3)
sqz.level_curve(ax_inset,V_the, xs,ps,xi,pi,"green",3)
sqz.level_curve(ax_inset,sigma, xs,ps,xi,pi,"red",3)

# Aggiunta di una colorbar personalizzata
cbar_ax = fig.add_axes([1.05, 0.1, 0.02, 0.8])  # [x0, y0, width, height]
# Aggiungere la colorbar
cbar = fig.colorbar(g, cax=cbar_ax)
cbar.ax.tick_params(labelsize=12)
plt.tight_layout()
plt.show()

Thermal noise
--------------

In [ ]:
X_grid = Y_grid = np.linspace(-5, 5, 100)  # (X,Y) grid on which we compute Wigner function
colors = [(0., 'darkblue'), (0.2, 'blue'), (0.4, 'skyblue'), (0.6, 'yellow'), (0.8, 'red'), (1., 'brown')] 
cmap = LinearSegmentedColormap.from_list('custom_cmap', colors)
#cmap = 'viridis'
vmin = 0.0    # colormap limits
vmax= 0.001
v = np.linspace(vmin,vmax,6)
vcenter = vmax/2
norm = TwoSlopeNorm(vmin=vmin,vcenter = vcenter, vmax=vmax)  # Crea l'istanza di normalizzazione con vmin e vmax
up_ins = 5   # iset limits
low_ins = -5

fig = plt.figure(figsize=(12, 8))
plt.subplot(2, 3, 1)    # ideal vacuum state
xs,ps,xi,pi = True, True, False, False   # whether to project onto this dimension or not
W_values_vac = sqz.wigner_proj((sigma_meas_off-identita), X_grid, Y_grid, xs, ps, xi, pi)   # wigner function projection on the desired dimensions on the grid
g = sqz.wigner_plot('', W_values_vac, cmap, X_grid, Y_grid, vmin, vmax, xs,ps,xi,pi)   # wigner function colorplot
ax_inset = sqz.inset_constructor_2(low_ins, up_ins)
sqz.level_curve(ax_inset,(sigma_meas_off-identita), xs,ps,xi,pi,"red",3)
T=3
V_the = (2*sqz.N(fs,T)+1)*identita
sqz.level_curve(ax_inset,V_the, xs,ps,xi,pi,"green",3)

In [ ]:
X_grid = Y_grid = np.linspace(-3, 3, 100)  # (X,Y) grid on which we compute Wigner function
colors = [(0., 'darkblue'), (0.2, 'blue'), (0.4, 'skyblue'), (0.6, 'yellow'), (0.8, 'red'), (1., 'brown')] 
cmap = LinearSegmentedColormap.from_list('custom_cmap', colors)
#cmap = 'viridis'
vmin = 0.0    # colormap limits
vmax= 0.01
v = np.linspace(vmin,vmax,6)
vcenter = vmax/2
norm = TwoSlopeNorm(vmin=vmin,vcenter = vcenter, vmax=vmax)  # Crea l'istanza di normalizzazione con vmin e vmax
up_ins = 2   # iset limits
low_ins = -2

fig = plt.figure(figsize=(8, 8))


plt.subplot(2, 2, 1)  # Reconstructed state xs ps
T = 0.15
xs,ps,xi,pi = True, True, False, False
V_the = (2*sqz.N(fs,T)+1)*identita
W_values = sqz.wigner_proj(sigma,X_grid, Y_grid, xs,ps,xi,pi)
g = sqz.wigner_plot('Reconstructed state', W_values, cmap, X_grid, Y_grid, vmin, vmax,xs,ps,xi,pi)
ax_inset = sqz.inset_constructor_2(low_ins, up_ins)
sqz.level_curve(ax_inset,identita, xs,ps,xi,pi,"blue",3)
sqz.level_curve(ax_inset,V_the, xs,ps,xi,pi,"green",3)
sqz.level_curve(ax_inset,sigma, xs,ps,xi,pi,"red",3)

plt.subplot(2, 2, 3)  # Reconstructed state xi pi
xs,ps,xi,pi = False, False, True, True
V_the = (2*sqz.N(fi,T)+1)*identita
W_values = sqz.wigner_proj(sigma, X_grid, Y_grid,xs,ps,xi,pi)
sqz.wigner_plot('Reconstructed state', W_values, cmap, X_grid, Y_grid, vmin, vmax,xs,ps,xi,pi)
ax_inset = sqz.inset_constructor_2(low_ins, up_ins)
sqz.level_curve(ax_inset,identita, xs,ps,xi,pi,"blue",3)
sqz.level_curve(ax_inset,V_the, xs,ps,xi,pi,"green",3)
sqz.level_curve(ax_inset,sigma, xs,ps,xi,pi,"red",3)

plt.subplot(2, 2, 2)  # Reconstructed state xs xi
xs,ps,xi,pi = True, False, True, False
V_the = (2*sqz.N(fs,T)+1)*identita
W_values = sqz.wigner_proj(sigma, X_grid, Y_grid,xs,ps,xi,pi)
sqz.wigner_plot('Reconstructed state', W_values, cmap,X_grid, Y_grid, vmin, vmax,xs,ps,xi,pi)
ax_inset = sqz.inset_constructor_2(low_ins, up_ins)
sqz.level_curve(ax_inset,identita, xs,ps,xi,pi,"blue",3)
sqz.level_curve(ax_inset,V_the, xs,ps,xi,pi,"green",3)
sqz.level_curve(ax_inset,sigma, xs,ps,xi,pi,"red",3)

plt.subplot(2, 2, 4)  # Reconstructed state ps pi
xs,ps,xi,pi = False, True, False, True
W_values = sqz.wigner_proj(sigma, X_grid, Y_grid,xs,ps,xi,pi)
sqz.wigner_plot('Reconstructed state', W_values, cmap, X_grid, Y_grid, vmin, vmax,xs,ps,xi,pi)
ax_inset = sqz.inset_constructor_2(low_ins, up_ins)
sqz.level_curve(ax_inset,identita, xs,ps,xi,pi,"blue",3)
sqz.level_curve(ax_inset,V_the, xs,ps,xi,pi,"green",3)
sqz.level_curve(ax_inset,sigma, xs,ps,xi,pi,"red",3)

# Aggiunta di una colorbar personalizzata
cbar_ax = fig.add_axes([1.05, 0.1, 0.02, 0.8])  # [x0, y0, width, height]
# Aggiungere la colorbar
cbar = fig.colorbar(g, cax=cbar_ax)
cbar.ax.tick_params(labelsize=12)
plt.tight_layout()
plt.show()